You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [1]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


Loading the configuration

In [2]:
from src.utils import load_config
import os
from config import DatasetConfig, TrainerConfig, BaselineConfig, GatrConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline
gatr_config: GatrConfig = config.gatr

Loading the dataset

In [3]:
from src.data import VesselDataModule


data = VesselDataModule(dataset_config)
print(f'Train size: {len(data.train_set)}')
print(f'Validation size: {len(data.val_set)}')
print(f'Test size: {len(data.test_set)}')
print(data.train_set[2])

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

Train size: 2999
Validation size: 599
Test size: 401
Data(pos=[8397, 3], wss=[8397, 3], pressure=[8397], face=[3, 16790], inlet_index=[214], label=Category.Single)


# TODO: analisi dati preliminare. Classi bilanciate? Statistiche sul dataset? Qualche plot figo.

# TODO: equivariance check! 

Testing

In [4]:
from src.trainer import VesselTrainer
from src.models import Gatr

trainer = VesselTrainer(trainer_config)


model = Gatr(gatr_config)
trainer.fit(model, data)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: danieleaffinita2000. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params | Mode 
--------------------------------------------------------------
0 | hsProjection    | EquiLinearLayer   | 576    | train
1 | backbone        | ModuleList        | 25.3 K | train
2 | finalProjection | Linear            | 76.8 K | train
3 | loss_fn         | BCEWithLogitsLoss | 0      | train
4 | train_accuracy  | BinaryAccuracy    | 0      | train
5 | val_accuracy    | BinaryAccuracy    | 0      | train
6 | test_accuracy   | BinaryAccuracy    | 0

/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/src/lib/geometricAlgebraOperations.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basis = torch.load(BI

Epoch 0: 100%|██████████| 750/750 [00:14<00:00, 50.24it/s, v_num=m2or, val/loss=0.0556, val/acc=0.987, val/f1=0.986, train/loss=0.172, train/acc=0.942, train/f1=0.941]

Metric val/loss improved. New best score: 0.056
Epoch 0, global step 750: 'val/loss' reached 0.05563 (best 0.05563), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=0-step=750.ckpt' as top 1


...

Epoch 18: 100%|██████████| 750/750 [00:17<00:00, 41.78it/s, v_num=m2or, val/loss=1.69e-5, val/acc=1.000, val/f1=1.000, train/loss=1.89e-6, train/acc=1.000, train/f1=1.000]

Epoch 18, global step 14250: 'val/loss' was not in top 1


Epoch 19: 100%|██████████| 750/750 [00:17<00:00, 42.79it/s, v_num=m2or, val/loss=4.16e-6, val/acc=1.000, val/f1=1.000, train/loss=1.13e-6, train/acc=1.000, train/f1=1.000]

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.000. Signaling Trainer to stop.
Epoch 19, global step 15000: 'val/loss' reached 0.00000 (best 0.00000), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=19-step=15000.ckpt' as top 1


Epoch 19: 100%|██████████| 750/750 [00:17<00:00, 42.76it/s, v_num=m2or, val/loss=4.16e-6, val/acc=1.000, val/f1=1.000, train/loss=1.13e-6, train/acc=1.000, train/f1=1.000]


In [5]:
trainer.test(model, data)

Restoring states from the checkpoint path at /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=19-step=15000.ckpt
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any us

Testing DataLoader 0: 100%|██████████| 101/101 [00:00<00:00, 102.20it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │            1.0            │
│          test/f1          │            1.0            │
│         test/loss         │   5.252342452877201e-05   │
└───────────────────────────┴───────────────────────────┘

Training Loop

In [6]:
from src.trainer import VesselTrainer
from src.models import BaselineTransformer

trainer = VesselTrainer(trainer_config)

model = BaselineTransformer(baseline_config)

trainer.fit(model, data)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
test/acc,▁
test/f1,▁
test/loss,▁
train/acc,▁▆▇▇▇███████████████
train/f1,▁▆▇▇▇███████████████
train/loss,█▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val/acc,▁▃▄▄▅▆██████████████
val/f1,▁▃▄▄▅▆██████████████
val/loss,█▇▄▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | encoder        | TransformerEncoder | 2.2 K  | train
1 | embedder       | Linear             | 272    | tr

Epoch 0: 100%|██████████| 750/750 [00:18<00:00, 39.70it/s, v_num=pdr5, val/loss=0.0518, val/acc=0.987, val/f1=0.986, train/loss=0.194, train/acc=0.921, train/f1=0.922]

Metric val/loss improved. New best score: 0.052
Epoch 0, global step 750: 'val/loss' reached 0.05180 (best 0.05180), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=0-step=750.ckpt' as top 1


Epoch 1: 100%|██████████| 750/750 [00:20<00:00, 37.20it/s, v_num=pdr5, val/loss=0.0438, val/acc=0.988, val/f1=0.988, train/loss=0.0576, train/acc=0.981, train/f1=0.982]

...

Epoch 9: 100%|██████████| 750/750 [00:20<00:00, 37.35it/s, v_num=pdr5, val/loss=0.0143, val/acc=0.995, val/f1=0.995, train/loss=0.000889, train/acc=1.000, train/f1=1.000] 

Epoch 9, global step 7500: 'val/loss' was not in top 1


Epoch 10: 100%|██████████| 750/750 [00:21<00:00, 35.55it/s, v_num=pdr5, val/loss=0.0173, val/acc=0.997, val/f1=0.997, train/loss=0.029, train/acc=0.994, train/f1=0.994]   

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.005. Signaling Trainer to stop.
Epoch 10, global step 8250: 'val/loss' was not in top 1


Epoch 10: 100%|██████████| 750/750 [00:21<00:00, 35.54it/s, v_num=pdr5, val/loss=0.0173, val/acc=0.997, val/f1=0.997, train/loss=0.029, train/acc=0.994, train/f1=0.994]


In [7]:
trainer.test(model, data)

Restoring states from the checkpoint path at /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=7-step=6000.ckpt
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use 

Testing DataLoader 0: 100%|██████████| 101/101 [00:01<00:00, 72.35it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.9975062608718872     │
│          test/f1          │     0.997481107711792     │
│         test/loss         │   0.010633878409862518    │
└───────────────────────────┴───────────────────────────┘

----